In [1]:
!pip install deep-translator

Defaulting to user installation because normal site-packages is not writeable


In [2]:
from deep_translator import GoogleTranslator
from include.utils import *
from transformers import BertForQuestionAnswering, BertTokenizer

import os
import pandas as pd
import torch

**Objetivo**

O objetivo deste script é demonstrar como usar um modelo BERT pré-treinado para responder a perguntas com base em um conjunto de dados. Este exemplo ilustra o processo de geração de respostas sem técnicas avançadas como fine-tuning ou Retrieval-Augmented Generation (RAG). Utilizamos um modelo de perguntas e respostas para mostrar a aplicação básica dessa técnica.

**Preparação dos dados**

Obtemos o arquivo parquet gerado e aplicamos uma amostragem opcional para limitar o tamanho do dataset, facilitando o teste inicial do fluxo de trabalho.

In [3]:
terminal_width = os.get_terminal_size().columns

diretorio_arquivos = '../arquivos'
parquet_file_path = os.path.join(diretorio_arquivos, 'trn.parquet')

# Carregar DataFrame do Parquet
dados = load_dataframe(parquet_file_path)

# Configurações de amostragem
batch_size = 50
use_sample = True

# Amostrar dados se necessário
if use_sample:
    dados = dados.sample(n=batch_size, random_state=42)

**Carregamento do Modelo e Tokenizador**

Carregando o modelo BERT pré-treinado e o tokenizador, configurando dois modelos: um para codificação de texto e outro para tarefas de perguntas e respostas.

In [4]:
# Carregar o modelo pré-treinado e o tokenizador
model_name = 'bert-large-uncased-whole-word-masking-finetuned-squad'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForQuestionAnswering.from_pretrained(model_name)

C:\Users\luiz.santos\AppData\Roaming\Python\Python312\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly

**Definição da Função de Perguntas e Respostas**

Define uma função para gerar respostas com base em uma pergunta e um contexto fornecido. Utiliza o modelo BERT para identificar a resposta mais provável no contexto.

In [5]:
# Inicializar tradutor
translator = GoogleTranslator(source='en', target='pt')

def obter_resposta(pergunta, contexto):
    # Preparar entrada para o modelo
    inputs = tokenizer.encode_plus(
        pergunta,
        contexto,
        return_tensors='pt',
        max_length=512,
        truncation=True
    )

    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']

    # Obter saída do modelo
    outputs = model(input_ids, attention_mask=attention_mask)

    # Encontrar índice inicial e final da resposta
    start_index = torch.argmax(outputs.start_logits)
    end_index = torch.argmax(outputs.end_logits) + 1

    # Obter tokens de resposta
    resposta_tokens = input_ids[0][start_index:end_index]
    resposta = tokenizer.decode(resposta_tokens, skip_special_tokens=True)

    return resposta

def traduzir_texto(texto):
    # Traduzir texto para o português
    traducao = translator.translate(texto)
    return traducao

pergunta = "Qual é a característica principal deste produto ?"

# Criar contextos a partir das amostras
contextos = [f"Title: {row['title']} Content: {row['content']}" for _, row in dados.iterrows()]

# Testar o modelo com as amostras
for _, row in dados.iterrows():
    titulo = row['title']
    contexto = row['content']

    if not contexto.strip():  # Verifica se o contexto está vazio
        print(f"Title: {titulo}")
        print("Context: Nenhum contexto fornecido.")
        print("Resposta: Nenhuma resposta encontrada.\n")
        print('-' * terminal_width)
        continue

    # Traduzir contexto para português
    contexto_pt = traduzir_texto(contexto)

    # Obter resposta
    resposta_en = obter_resposta(pergunta, contexto)

    # Traduzir resposta para português
    resposta_pt = traduzir_texto(resposta_en)

    # Imprimir resultado formatado
    print(f"Title: {titulo}")
    print(f"Contexto em inglês: {contexto}")
    print(f"Contexto traduzido: {contexto_pt}")
    print(f"Resposta em inglês: {resposta_en}")
    print(f"Resposta traduzida: {resposta_pt}\n")
    print('-' * terminal_width)

Title: Yellow Box Women's Yulisa Flip Flop Sandal
Context: Nenhum contexto fornecido.
Resposta: Nenhuma resposta encontrada.

----------------------------------------------------------------------------------------------------------------------------------------------------
Title: Nine West Women's Eastnor Sandal
Context: Nenhum contexto fornecido.
Resposta: Nenhuma resposta encontrada.

----------------------------------------------------------------------------------------------------------------------------------------------------
Title: Roma Costume Women's 1 Piece Santa's Envy-Red White
Context: Nenhum contexto fornecido.
Resposta: Nenhuma resposta encontrada.

----------------------------------------------------------------------------------------------------------------------------------------------------
Title: 1 Box of 100 Sony 1.55v Silver Oxide Watch Batteries 377 SR626SW
Contexto em inglês: 100 Sony 377 Watch Batteries Silver Oxide, Mercury Free
Contexto traduzido: 100 Sony

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Title: Winning Angels: The 7 Fundamentals of Early Stage Investing
Contexto em inglês: "Winning Angels is a superbly organised and invaluable handbook to start-up investing. You will examine the shape of a typical deal, from chance contact at a networking group to flotation on the stock market. And all the time, you will be told to ask questions. What does my intuition tell me? Do I have experience in this area? Is there a clear product and revenue possibility? And you will learn about leaving people wanting to deal with you in the future. Very wise--that next one could be the real thing."&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Amazon.co.uk&nbsp;"An insightful and fact-filled primer on how to ferret out great opportunities."&nbsp;&#xBF; Richard Kramlich, angel in Apple Computer, and partner at Venture Capitalist firm, New Enterprise Associates &nbsp;"Entrepreneurs would do well to read and reflect on the contents of the book before they seek funding."&nbsp;&#xBF; Peter Crisp, foundin

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Title: Porter Cable/Dewalt Compressor OEM CYLINDER Repair Kit # N036517
Contexto em inglês: Compatible with the Following Models: er Cable: CF2600 Type 2,CPFAC2040P Type 2,CPFAC2040P Type 3,CPFAC2600P Type 2,CPFAC2600P Type 3,CFBN220 A,CFBN220 Type 1 A,CFFN250B Type 2 A,CFFN250B Type 3 A,CFFN250T,E2002 A,E2002 Type 1 A,CFBN125A Type 2 A,CFBN125A Type 3 A,CFBN125N A,CFBN125N Type 1 A,CFBN200A Type 2 A,CFBN200A Type 3 A,CFBN217A Type 1 A,CFBN217A Type 2 A,C2005,C2006,C2006 Type 2,C2006 Type 3,C2000 A,C2000 Type 1 A,C2001-WK A,C3001 A,C30011 A,C3001 Type 1 A,C3001 Type 2 A,CFFN250N Type 2,C2025,C2005 Type 2,C2005 Type 3,CFFN250N 2.0,CFFN251T,CF1400,CF2600,CF2600 Type 1,CPFAC2040P,CPFAC2040P Type 1,CPFAC2600P,CPFAC2600P Type 1,CFFN250B Type 1 A,CFBN200A A,C2002-WK,CPFAC2600P-W A,CPFAC2600P-W Type 1 A,C6001,C2006 Type 4 C2002 Type 7,C2002 Type 5 C2002 Type 4,C2000-WK (Type 0) C2000-WK (Type 1),C2004-WK Type 1 C2004-WK Type 3,Delta: CP503 Type 1,CP503 Type 2,CP503 Type 3,CP503,DeVilbiss: CPF